[Crypto Fear & Greed Index - Bitcoin Sentiment - Alternative.me](https://alternative.me/crypto/fear-and-greed-index/#api)

In [1]:
import requests
import json
import time

In [2]:
#제공해주는 api 주소(url)을 통해 json형식의 데이터를 가져오고 이를 파싱해
#오늘(day), 어제(yester), 이틀전(twodaysago), 주평균(week), 월평균(month) 공포/탐욕 지수를 가져오기

url = "https://api.alternative.me/fng/?limit="

def fear_day():
    _url = url+"1"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]
    
    return data[0]["value"]
                
def fear_yester():
    _url = url+"2"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]

    return data[1]["value"]
    
def fear_twodaysago():
    _url = url+"3"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]

    return data[2]["value"]

def fear_week():
    _url = url+"7"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]
    
    sum = 0
    for index, value in enumerate(data):
        sum += int(value["value"])
        
    return sum/7
    
def fear_month():
    _url = url+"30"
    res = requests.request("GET", _url);
    
    parsed = json.loads(res.text)
    data = parsed["data"]

    sum = 0
    for index, value in enumerate(data):
        sum += int(value["value"])
    
    return sum/30

In [3]:
fear_yester()

'15'

In [4]:
fear_day()

'18'

In [5]:
fear_twodaysago()

'24'

In [6]:
url1="https://api.alternative.me/fng/?limit=10&format=csv&date_format=us"

In [7]:
print(url1)

https://api.alternative.me/fng/?limit=10&format=csv&date_format=us


In [8]:
url2 = "https://api.alternative.me/fng/?limit=100"
print(url2)

https://api.alternative.me/fng/?limit=100


In [9]:
# 주소 뒤에 여러 옵션 파라메터를 추가할 수 있다. 

# - 데이터 개수 -> limit= n으로 설정. 0으로 입력시 가능한한 많은 데이터를 불러온다 
# - 포멧 -> json, csv 선택 가능. 기본은 json 
# - 날짜 형식 -> json의 경우 timestamp형식으로 불러와진다. kr을 입력시 2021/12/12와 같이 나온다 

# 다음은 api를 이용해서 불러올 수 있는 최대한의 데이터를 불러와서 fng라는 판다스 Series에 데이터를 저장

# date_format을 이용하지 않고 timestamp를 변환하는 식으로 코드를 짰다. 

import requests
import datetime
import pandas as pd

url = ' https://api.alternative.me/fng/?limit=0'
data = requests.get(url).json()
value = []
time = []
for i in data['data']:
    value.append(i['value'])
    time.append(datetime.datetime.fromtimestamp(int(i['timestamp'])).strftime('%Y-%m-%d'))
value = value[::-1]
time = pd.to_datetime(time[::-1])
fng = pd.Series(value,time,name='fng')

In [10]:
df_fng = fng.reset_index(name='value')

In [11]:
df_fng = df_fng.rename(columns={'index':'datetime'})

In [12]:
url = ' https://api.alternative.me/fng/?limit=0'
data = requests.get(url).json()
value = []
time = []
for i in data['data']:
    value.append(i['value'])
    time.append(i['timestamp'])
value = value[::-1]
time = time[::-1]
tv = pd.Series(value,time,name='tv')

In [13]:
df_greed = tv.reset_index(name='value')

In [14]:
df_greed = df_greed.rename(columns={'index':'timestamp'})

In [15]:
df_greed

,timestamp,value
0,1517443200,30
1,1517529600,15
2,1517616000,40
3,1517702400,24
4,1517788800,11
...,...,...
1429,1641168000,29
1430,1641254400,23
1431,1641340800,24
1432,1641427200,15


In [16]:
df_greed = df_greed.drop(["value"], axis=1)

In [17]:
df_greed = pd.concat([df_greed,df_fng],axis=1)

In [18]:
url = ' https://api.alternative.me/fng/?limit=0'
data = requests.get(url).json()
value = []
value_class = []
for i in data['data']:
    value.append(i['value'])
    value_class.append(i['value_classification'])
value = value[::-1]
value_class = value_class[::-1]
tv = pd.Series(value,value_class,name='tv')

In [19]:
df_g = tv.reset_index(name='value')

In [20]:
df_g = df_g.rename(columns={'index':'value_class'})

In [21]:
df_g = df_g.drop(["value"], axis=1)

In [22]:
df_greed = pd.concat([df_greed,df_g],axis=1)

In [23]:
df_greed

,timestamp,datetime,value,value_class
0,1517443200,2018-02-01,30,Fear
1,1517529600,2018-02-02,15,Extreme Fear
2,1517616000,2018-02-03,40,Fear
3,1517702400,2018-02-04,24,Extreme Fear
4,1517788800,2018-02-05,11,Extreme Fear
...,...,...,...,...
1429,1641168000,2022-01-03,29,Fear
1430,1641254400,2022-01-04,23,Extreme Fear
1431,1641340800,2022-01-05,24,Extreme Fear
1432,1641427200,2022-01-06,15,Extreme Fear


In [24]:
df_greed.to_csv("CryptoFear.csv", index = False)

In [25]:
pd.read_csv("CryptoFear.csv")

,timestamp,datetime,value,value_class
0,1517443200,2018-02-01,30,Fear
1,1517529600,2018-02-02,15,Extreme Fear
2,1517616000,2018-02-03,40,Fear
3,1517702400,2018-02-04,24,Extreme Fear
4,1517788800,2018-02-05,11,Extreme Fear
...,...,...,...,...
1429,1641168000,2022-01-03,29,Fear
1430,1641254400,2022-01-04,23,Extreme Fear
1431,1641340800,2022-01-05,24,Extreme Fear
1432,1641427200,2022-01-06,15,Extreme Fear
